## import

In [ ]:
import os
import shutil

import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image

## image preparation

In [ ]:
# conversion
files = os.listdir('archive/videos')
files.sort(key = len)

level = ['BL', 'CD', 'ED', 'FD', 'MD', 'ND', 'PD', 'RD']
strsr = [2, 3, 4, 0, 5, 2, 2, 2]
txt = np.genfromtxt(fname = 'archive/FD.txt')

lbl = np.array([strsr, ] * len(txt))
lbl[:, 3] = np.array(txt).astype(int)

count = 0
for i in range(len(files)):
    vidcap = cv2.VideoCapture('archive/videos/' + files[i])
    success, image = vidcap.read()

    while success:
        cv2.imwrite('archive/lbl_img/' + str(count).zfill(7) + '_' +
                    files[i][-6:-4] + '_' +
                    level[(int(files[i][0:-7]) - 1) % 8] + '_' +
                    str(lbl[int(files[i][-6:-4]) - 1, (int(files[i][0:-7]) - 1) % 8]) + '.jpg',
                    image)
        success, image = vidcap.read()
        count += 1

In [ ]:
# lable
files = os.listdir('archive/lbl_img')
#df      = pd.DataFrame(columns=['row', 'file name', 'subject id', 'level', 'type lable', 'stressor lable', 'frame counter'])
#g       = files[0][11:13]
#counter = 1

for i in range(1525180, len(files)):
    tmp = int(files[i][14:15])
    lvl = files[i][11:13]

    if lvl != g:
        counter = counter + 1
        g = lvl
        print(counter)

    if tmp > 2:
        typee = 1
        stressor = tmp - 2
    elif tmp == 2:
        typee = 0
        stressor = 0
    elif tmp == 1:
        typee = 2
        stressor = 4
    elif tmp == 0:
        typee = 3
        stressor = 5

    new_row = {
        'row':            int(files[i][0:7]),
        'file name':      files[i],
        'subject id':     int(files[i][8:10]),
        'level':          lvl,
        'type lable':     typee,
        'stressor lable': stressor,
        'frame counter':  counter
        }
    df = df.append(new_row, ignore_index = True)


In [ ]:
# save
df.to_csv('video dataset/lables/df.csv')

In [ ]:
# load
df = pd.read_csv('video dataset/lables/df.csv')
df = df.drop(columns = ['Unnamed: 0'])

In [ ]:
# find faces
classifier = load_model('checkpoint mine classifier')
n = os.listdir('archive/lbl_img/')
lables = []

for i in range(1501305, len(n)):
    img = []

    train_igg = image.load_img('archive/lbl_img/' + n[i], color_mode = "grayscale")
    train_img = image.img_to_array(train_igg)
    train_img /= 255.0
    img.append(train_img[:, 64:576])
    img = np.array(img)
    lbl = classifier.predict(img)
    lables.append(lbl.reshape(4))
    print(i)

In [ ]:
# check
i = 5
train_igg = image.load_img('archive/lbl_img/' + my_lbl['file name'][i], color_mode = "grayscale")

a = np.ones(512)
b = np.ones(512)

plt.figure(figsize = (20, 10))
plt.imshow(image.img_to_array(train_igg)[:, 64:576, 0])

plt.plot(my_lbl['Ymin'][i] * b)
plt.plot(my_lbl['Ymax'][i] * b)
plt.plot(my_lbl['Xmin'][i] * a, range(np.shape(train_igg)[0]))
plt.plot(my_lbl['Xmax'][i] * a, range(np.shape(train_igg)[0]))
plt.show()

In [ ]:
# save as df
lables = np.array(lables).astype(int)
my_lbl = pd.DataFrame(list(zip(n, lables[:, 0], lables[:, 1], lables[:, 2], lables[:, 3])), columns = ['file name',
                                                                                                       'Xmin', 'Xmax',
                                                                                                       'Ymin', 'Ymax'])
my_lbl.to_csv('video dataset/lables/my_lbl.csv')

In [ ]:
# load
my_lbl = pd.read_csv('video dataset/lables/my_lbl.csv')
#my_lbl = df.drop(columns=['Unnamed: 0'])

In [ ]:
# find frame
h = my_lbl['Ymax'] - my_lbl['Ymin']
w = my_lbl['Xmax'] - my_lbl['Xmin']

h = np.array(h)
w = np.array(w)

n, bins, patches = plt.hist(h, bins = 20)
plt.show()
n, bins, patches = plt.hist(w, bins = 20)
plt.show()

In [ ]:
# prepare frames
cte = my_lbl.copy()

cte['height'] = h
cte['width'] = w

cte['Y1'] = ''
cte['Y2'] = ''
cte['X1'] = ''
cte['X2'] = ''

idx = np.where(cte['height'] < 400)
idx = np.array(idx).ravel()
cte['Y1'][idx] = round(cte['Ymin'][idx] - (400 - cte['height'][idx]) / 2).astype(int)
cte['Y2'][idx] = round(cte['Ymax'][idx] + (400 - cte['height'][idx]) / 2).astype(int)

idx = np.where(cte['height'] > 400)
idx = np.array(idx).ravel()
cte['Y1'][idx] = round(cte['Ymin'][idx] + (cte['height'][idx] - 400) / 2).astype(int)
cte['Y2'][idx] = round(cte['Ymax'][idx] - (cte['height'][idx] - 400) / 2).astype(int)

idx = np.where(cte['height'] == 400)
idx = np.array(idx).ravel()
cte['Y1'][idx] = round(cte['Ymin'][idx]).astype(int)
cte['Y2'][idx] = round(cte['Ymax'][idx]).astype(int)

idx = np.where(cte['width'] < 300)
idx = np.array(idx).ravel()
cte['X1'][idx] = round(cte['Xmin'][idx] - (300 - cte['width'][idx]) / 2).astype(int)
cte['X2'][idx] = round(cte['Xmax'][idx] + (300 - cte['width'][idx]) / 2).astype(int)

idx = np.where(cte['width'] > 300)
idx = np.array(idx).ravel()
cte['X1'][idx] = round(cte['Xmin'][idx] + (cte['width'][idx] - 300) / 2).astype(int)
cte['X2'][idx] = round(cte['Xmax'][idx] - (cte['width'][idx] - 300) / 2).astype(int)

idx = np.where(cte['width'] == 300)
idx = np.array(idx).ravel()
cte['X1'][idx] = round(cte['Xmin'][idx]).astype(int)
cte['X2'][idx] = round(cte['Xmax'][idx]).astype(int)

cte['X1'] = pd.to_numeric(cte['X1'])
idx = np.where(cte['X1'] <= 0)
idx = np.array(idx).ravel()
cte['X2'][idx] = 301
cte['X1'][idx] = 1

cte['X2'] = pd.to_numeric(cte['X2'])
idx = np.where(cte['X2'] >= 512)
idx = np.array(idx).ravel()
cte['X1'][idx] = 211
cte['X2'][idx] = 511

cte['Y1'] = pd.to_numeric(cte['Y1'])
idx = np.where(cte['Y1'] <= 0)
idx = np.array(idx).ravel()
cte['Y2'][idx] = 401
cte['Y1'][idx] = 1

cte['Y2'] = pd.to_numeric(cte['Y2'])
idx = np.where(cte['Y2'] >= 480)
idx = np.array(idx).ravel()
cte['Y1'][idx] = 79
cte['Y2'][idx] = 479

In [ ]:
# save
cte = cte.drop(columns = ['Xmin', 'Xmax', 'Ymin', 'Ymax', 'height', 'width'])
cte.to_csv('video dataset/lables/cte.csv')

In [ ]:
# load
cte = pd.read_csv('video dataset/lables/cte.csv')
cte = cte.drop(columns = ['Unnamed: 0'])

In [ ]:
# add lables to main df
df['X1'] = cte['X1'].copy()
df['X2'] = cte['X2'].copy()
df['Y1'] = cte['Y1'].copy()
df['Y2'] = cte['Y2'].copy()

In [ ]:
# remove FD
idx = np.where((df['level'] == 'PD') | (df['level'] == 'FD'))
df = df.drop(np.array(idx).ravel())

In [ ]:
# number of frames
frames = df['frame counter'].tolist()
frames = [int(i) for i in frames]
output = []

for x in frames:
    if x not in output:
        output.append(x)

rep = np.zeros((len(output), 2))
output.sort()

for i in range(len(output)):
    rep[i, 0] = output[i]
    rep[i, 1] = frames.count(output[i])

rep = rep.astype(int)

n, bins, patches = plt.hist(rep[:, 1], bins = 20)
plt.show()

print(min(rep[:, 1]))
print(rep)

In [ ]:
# separate short videos
idx = np.where((df['frame counter'] == 209) | (df['frame counter'] == 322))
df_short = df.iloc[np.array(idx).ravel()]

A = [i for i in list(range(len(df))) if i not in np.array(idx).ravel()]
df_long = df.iloc[A]

output.remove(209)
output.remove(322)

d = np.where((rep[:, 0] == 209) | (rep[:, 0] == 322))
rep = np.delete(rep, d[0], 0)

In [ ]:
#check frame
h = df_long['Y2'] - df_long['Y1']
w = df_long['X2'] - df_long['X1']

h = np.array(h)
w = np.array(w)

n, bins, patches = plt.hist(h)
plt.show()
n, bins, patches = plt.hist(w)
plt.show()

In [ ]:
# prepare frames
sequence = 80
seq_numb = 20
l = sequence * seq_numb
counter = 1
tl = df_long['type lable'].tolist()
sl = df_long['stressor lable'].tolist()
fn = df_long['file name'].tolist()
y1 = df['Y1'].tolist()
y2 = df['Y2'].tolist()
x1 = df['X1'].tolist()
x2 = df['X2'].tolist()

for i in range(len(output)):
    idx = np.where(df_long['frame counter'] == rep[i, 0])
    idx = np.array(idx).ravel()
    minus = round((l - rep[i, 1]) / (seq_numb + 1))

    for j in range(seq_numb):
        os.mkdir('video dataset/videos/' + str(counter).zfill(4) + '_' + str(tl[idx[0]]) + '_' + str(sl[idx[0]]))

        for k in range(sequence):
            ii = idx[(j + 1) * minus + j * sequence + k]
            image = imageio.imread('archive/lbl_img/' + fn[ii])
            image = image[int(y1[ii]):int(y2[ii]), (int(x1[ii]) + 64):(int(x2[ii]) + 64), :]
            imageio.imwrite('video dataset/videos/' + str(counter).zfill(4) + '_' + str(tl[idx[0]]) + '_' + str(
                sl[idx[0]]) + '/' + str(k) + '.jpg', image)

        counter = counter + 1

# data aug

In [ ]:
files = os.listdir('I:/poorya/datasets/video dataset/videos')
counter = 0

for i in range(len(files)):
    for j in range(2):
        os.mkdir('I:/poorya/datasets/video dataset/videos lts/' + str(counter).zfill(4) + files[i][4:])
        for k in range(40):
            shutil.copy2('I:/poorya/datasets/video dataset/videos/' + files[i] + '/' + str(j * 40 + k) + '.jpg',
                         'I:/poorya/datasets/video dataset/videos lts/' + str(counter).zfill(4) + files[i][
                                                                                                  4:] + '/' + str(k) + '.jpg')
        counter = counter + 1